In [1]:
from astropalmerio.mc.sampling import sample_asym_norm
from astropalmerio.mc.realizations import MC_realization
from astropalmerio.mc.MC_var import MC_var
from astropalmerio.mc.utils import unbinned_empirical_cdf, binned_CDFs_from_realizations, quantiles
from astropalmerio.mc.visualization import plot_CDF_with_bounds, plot_ECDF, add_arrows_for_limits
from astropalmerio.galaxies.properties import Av_from_Balmer_decrement

import cmasher as cmr

import time
import logging
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import sys
from pathlib import Path


log = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.INFO,
                    format='%(asctime)s %(levelname)s [%(name)s] %(message)s')
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.getLogger('PIL').setLevel(logging.WARNING)

ROOT_DIR = Path('/Users/palmerio/Code_projects/astropalmerio/dev/')

plt.style.use('paper')
plt.style.use('kraken')

## How to calculate the Star-Formation Rate (SFR) of a galaxy using $H\alpha$
In this example, we will show how to propagate the uncertainties from the measurement of two emission lines ($H\alpha$ and $H\beta$) onto a final physical property of a galaxy (SFR).
For the purpose of this example, we will be studying a distant galaxy for which we have obtained an optical spectrum.
This has allowed us to measure the fluxes of the $H\alpha$ and $H\beta$ emission lines, yielding the following result:

- $F_{H\alpha} = (30 \pm 5) \times 10^{-17}$ erg/cm2/s
- $F_{H\beta} = (8 \pm 3) \times 10^{-17}$ erg/cm2/s

The errors here are due solely to measurement uncertainty, which is a random process that we assume to yield gaussian errors (we will neglect systematic errors for the purpose of this example).
Let us assume that the fluxes measured are corrected for Balmer absorption, Galactic foreground extinction and that they only require correction for intrinsic attenuation.

### Correcting for intrinsic attenuation
#### Determining host $A_{\rm v}$ using the Balmer decrement
We can estimate the amount of attenuation in the host galaxy by measuring the Balmer decrement, that is the ratio of the $H\alpha$ to $H\beta$ flux:

- $R^{\rm intr}_{H\alpha/H\beta} = \frac{F^{\rm intr}_{H\alpha}}{F^{\rm intr}_{H\beta}}=2.86$

We know that this ratio should be equal to $2.86$ according to the scenario of case B recombination at 10 000 K ([Osterbrock+06](https://ui.adsabs.harvard.edu/abs/2006agna.book.....O/abstract)).
We can use our _observed_ ratio to infer how much attenuation there is:

- $R^{\rm obs}_{H\alpha/H\beta} =\frac{F^{\rm obs}_{H\alpha}}{F^{\rm obs}_{H\beta}} = \frac{F^{\rm intr}_{H\alpha} \times 10^{-0.4 A_{\rm v}\xi(\lambda_{H\alpha})}}{F^{\rm intr}_{H\beta}\times 10^{-0.4 A_{\rm v}\xi(\lambda_{H\beta})}} = R^{\rm intr}_{H\alpha/H\beta} \times 10^{-0.4 A_{\rm v}(\xi(\lambda_{H\alpha})-\xi(\lambda_{H\beta}))} = 2.86 \times 10^{-0.4 A_{\rm v}(\xi(\lambda_{H\alpha})-\xi(\lambda_{H\beta}))}$

Assuming a certain form for the extinction curve $\xi(\lambda)$ from [Pei+99](https://ui.adsabs.harvard.edu/abs/1992ApJ...395..130P/abstract), we can rearrange the equation to get the visible band attenuation $A_{\rm v}$:

- $A_{\rm v} = -2.5 \frac{\log(R^{\rm obs}_{H\alpha/H\beta}/2.86)}{\xi(\lambda_{H\alpha})-\xi(\lambda_{H\beta})}$

In [2]:
F_Ha = 30
F_Ha_unc = 5
F_Hb = 8
F_Hb_unc = 3

F_Ha_MC = MC_realization(F_Ha, F_Ha_unc)
F_Hb_MC = MC_realization(F_Hb, F_Hb_unc, val_min=0)

ratio_obs_MC = F_Ha_MC/F_Hb_MC
logging.getLogger('astropalmerio.galaxies.extinction').setLevel(logging.DEBUG)
logging.getLogger('astropalmerio.galaxies.properties').setLevel(logging.DEBUG)

Av_MC = Av_from_Balmer_decrement(F_Ha=F_Ha,
                                 F_Hb=F_Hb)
Av_MC

2023-03-10 11:13:02,081 DEBUG [astropalmerio.galaxies.properties] Estimating Av from:
Observed Ha/Hb = 3.75
Theoretical Ha/Hb = 2.847
Av = [0.84972966]


{'Ha/Hb': array([0.84972966])}

In [3]:
import astropalmerio.galaxies.extinction as ext
ext.Pei92_SMC(6500), ext.Pei92_SMC(4800)
np.mean([0.3])

0.3

#### Correcting the observed $H\alpha$ flux
And finally the corrected (intrinsic) $H\alpha$ flux:

- $F^{\rm intr}_{H\alpha} = F^{\rm obs}_{H\alpha} \times 10^{+0.4 A_{\rm v} \xi(\lambda_{H\alpha})}$

We can now use the intrinsic $F_{H\alpha}$ to estimate the SFR with the relation of [Kennicutt+98](https://ui.adsabs.harvard.edu/abs/1998ARA%26A..36..189K/abstract):

- ${\rm SFR} = 7.9\times10^{-42}\, L_{H\alpha}~[\rm M_{\odot}\,yr^{-1}]$

SFR = star_formation_rate()